In [1]:
import datetime
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import random
import sys
import time
import subprocess
from pathlib import Path
import speechpy

from concurrent.futures import ThreadPoolExecutor, wait, as_completed
from torch import nn, optim
from tqdm import tqdm

In [2]:
TEST_videos = pd.read_csv('/home/jupyter/results_TEST/TEST_PREDS_2020-03-12_12-20-58_efficientnet-b4-radam-lookahead-baseline-undersampling_995090_ep3.csv').iloc[:, 1:]
DEV_videos =  pd.read_csv('/home/jupyter/results_DEV/DEV_PREDS_2020-03-03_07-25-04_efficientnet-b4-radam-lookahead-baseline_994682_ep4.csv').iloc[:, 1:]

In [3]:
TEST_names = TEST_videos.iloc[:, 0].values
DEV_names = DEV_videos.iloc[:, 0].values

### TEST

In [4]:
test_dir = '/home/jupyter/data/videos'
test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4" and x in TEST_names])
len(test_videos)

1143

In [5]:
windows = [50000, 100000, 150000, 200000]
def getFeatures(x, sr):
    features = []
    
    features.append(np.mean(x))
    features.append(np.min(x))
    features.append(np.max(x))
    features.append(np.std(x))
    features.append(np.mean(np.diff(x)))
    features.append(np.max(abs(x)))
    features.append(np.min(abs(x)))
    
    
    # Example of staching frames
    frames = speechpy.processing.stack_frames(x, sampling_frequency=sr, frame_length=0.020, frame_stride=0.01, filter=lambda x: np.ones((x,)),
             zero_padding=True)

    # Example of extracting power spectrum
    power_spectrum = speechpy.processing.power_spectrum(frames, fft_points=512)
    #print('power spectrum shape=', power_spectrum.shape)

    ############# Extract MFCC features #############
    mfcc = speechpy.feature.mfcc(x, sampling_frequency=sr, frame_length=0.020, frame_stride=0.01,
                 num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    mfcc_cmvn = speechpy.processing.cmvnw(mfcc,win_size=301,variance_normalization=True)
    #print('mfcc(mean + variance normalized) feature shape=', mfcc_cmvn.shape)

#     mfcc_feature_cube = speechpy.feature.extract_derivative_feature(mfcc)
#     print('mfcc feature cube shape=', mfcc_feature_cube.shape)

    ############# Extract logenergy features #############
    logenergy = speechpy.feature.lmfe(x, sampling_frequency=sr, frame_length=0.020, frame_stride=0.01,
                 num_filters=40, fft_length=512, low_frequency=0, high_frequency=None)
    logenergy_feature_cube = speechpy.feature.extract_derivative_feature(logenergy)
    #print('logenergy features=', logenergy.shape)
    
    features.append(np.mean(power_spectrum))
    features.append(np.mean(mfcc_cmvn))
    features.append(np.mean(logenergy))
    
    features.append(np.median(power_spectrum))
    features.append(np.median(mfcc_cmvn))
    features.append(np.median(logenergy))
    
    features.append(np.std(power_spectrum))
    features.append(np.std(mfcc_cmvn))
    features.append(np.std(logenergy))
    #features.append(power_spectrum.mean())
    
#     windows = [50000, 100000, 150000, 200000]
#     for i in range(1, len(windows)):
#         lb, rb = windows[i-1], windows[i]
#         features.append(np.mean(x[lb:rb]))
#         features.append(np.min(x[lb:rb]))
#         features.append(np.max(x[lb:rb]))
#         features.append(np.std(x[lb:rb]))
#         features.append(np.mean(np.diff(x[lb:rb])))
#         features.append(np.max(abs(x[lb:rb])))
#         features.append(np.min(abs(x[lb:rb])))
    return features

In [6]:
import pickle
import librosa
import os
import hashlib

model = pickle.load(open('xgboost_model.pkl', "rb"))
def predict_on_video(model, video_path):
    try:
        filename =  hashlib.sha1(bytes(video_path, 'utf8')).hexdigest() + '.wav'
        command = 'ffmpeg-git-20191209-amd64-static/ffmpeg -i ' + video_path + ' -ab 192000 -ac 2 -ar 44100 -vn ' + filename
        subprocess.call(command, shell=True)
        x , sr = librosa.load(filename)
        features = getFeatures(x, sr)
        os.remove(filename)
        return model.predict_proba(features)[0][0]
    
    except Exception as e:
        print("Prediction error on video %s: %s" % (video_path, str(e)))

    return 0.5

In [7]:
from concurrent.futures import ThreadPoolExecutor
import pickle

def predict_on_video_set(model, videos, num_workers):
    def process_file(i):
        filename = videos[i]
        y_pred = predict_on_video(model, os.path.join(test_dir, filename))
        return y_pred

    with ThreadPoolExecutor(max_workers=num_workers) as ex:
        predictions = ex.map(process_file, range(len(videos)))

    return list(predictions)

In [8]:
speed_test = True
# you have to enable this manually

In [9]:
import multiprocessing
if speed_test:
    start_time = time.time()
    speedtest_videos = test_videos[:5]
    predictions = predict_on_video_set(model, speedtest_videos, num_workers=2*multiprocessing.cpu_count())
    elapsed = time.time() - start_time
    print("Elapsed %f sec. Average per video: %f sec." % (elapsed, elapsed / len(speedtest_videos)))

1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
Elapsed 5.553406 sec. Average per video: 1.110681 sec.


In [10]:
predictions = np.array(predict_on_video_set(model, test_videos, num_workers=2*multiprocessing.cpu_count()))

1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 1001220618 220618 441 220.0
 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1 463 441 220.0
Prediction error on video /home/jupyter/data/videos/apddumqkmt.mp4: integer division or modulo by zero
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
Prediction error on video /home/jupyter/data/videos/arebkvnjtb.mp4: Input signal length=0 is too small to resample from 44100->22050
Prediction error on video /home/jupyter/data/videos/aruyyclubn.mp4: Input signal length=0 is too small to resample from 44100->22050
1006 221559 441 220.0
1001 220618 441 220.0

/usr/local/lib/python3.5/dist-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


1001 220618 441 220.0
Prediction error on video /home/jupyter/data/videos/clkxgilfnl.mp4: 
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 1001220618 441 220.0
 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 22

1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 1001220618 441  220.0
220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.01001 
220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1002 22067

1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1006 221559 441 220.0
1001 22061

In [11]:
submission_df = pd.DataFrame({"filename": test_videos, "label": predictions})
submission_df['label'] = submission_df['label'].astype(float)
submission_df.to_csv("/home/jupyter/TEST.csv", index=False)

### DEV

In [12]:
test_dir = '/home/jupyter/data/videos'
test_videos = sorted([x for x in os.listdir(test_dir) if x[-4:] == ".mp4" and x in DEV_names])
len(test_videos)

1140

In [13]:
predictions = np.array(predict_on_video_set(model, test_videos, num_workers=2*multiprocessing.cpu_count()))

1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 22061

1006 221559 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1006 221559 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
10011001 220618 441 220.0
 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 22061

1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 22061

1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1002 220672 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0
1001 220618 441 220.0


In [14]:
submission_df = pd.DataFrame({"filename": test_videos, "label": predictions})
submission_df['label'] = submission_df['label'].astype(float)
submission_df.to_csv("/home/jupyter/DEV.csv", index=False)